In [1]:
lr = 2e-5 
epoch = int(2e3) 
batch_size = 100
layer_widths = [784, 600, 500, 400, 300, 200, 100, 50, 10, 2]

In [2]:
import numpy as np; np.random.seed(0)
import tensorflow.compat.v1 as tf; tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train, x_test = x_train.reshape((-1, 784)), x_test.reshape((-1, 784))
x_train, x_test = x_train.astype(np.float32), x_test.astype(np.float32)

In [4]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, (None, 784)) # (?, 784)

init_w = tf.keras.initializers.VarianceScaling(seed=0)
init_b = tf.keras.initializers.zeros()

w_shapes = []
previous = layer_widths[0] 
for layer_width in layer_widths[1:]:
    w_shapes.append((previous, layer_width))
    previous = layer_width

for i, (pre, layer_width) in enumerate(w_shapes):
    w_name = f'w{i}_encoder'
    b_name = f'b{i}_encoder'
    w = tf.get_variable(w_name, (pre, layer_width), tf.float32, init_w)
    b = tf.get_variable(b_name, (layer_width,), tf.float32, init_b)
    if i==0:
        x1 = tf.nn.selu(x @ w + b)
    elif i!=len(w_shapes)-1:
        x1 = tf.nn.selu(x1 @ w + b)
    else:
        hidden = tf.nn.sigmoid(x1 @ w + b)        
        
for i, (pre, layer_width) in enumerate(w_shapes[::-1]):
    w_name = f'w{len(w_shapes)-1-i}_decoder'
    b_name = f'b{len(w_shapes)-1-i}_decoder'
    w = tf.get_variable(w_name, (layer_width, pre), tf.float32, init_w)
    b = tf.get_variable(b_name, (pre,), tf.float32, init_b)
    if i==0:
        x1 = tf.nn.selu(hidden @ w + b)
    elif i!=len(w_shapes)-1:
        x1 = tf.nn.selu(x1 @ w + b)
    else:
        x_recon = tf.nn.sigmoid(x1 @ w + b)

binary_entropy = tf.keras.losses.binary_crossentropy(x, x_recon)
loss = tf.reduce_mean(binary_entropy)

opt = tf.train.AdamOptimizer(lr)
train = opt.minimize(loss)

init = tf.global_variables_initializer()

[(784, 600), (600, 500), (500, 400), (400, 300), (300, 200), (200, 100), (100, 50), (50, 10), (10, 2)]
()


In [ ]:
with tf.Session() as sess:
    
    sess.run(init)
    
    loss_trace = []

    for i in range(epoch):
        
        idx = np.arange(x_train.shape[0])
        np.random.shuffle(idx)
        
        x_epoch = x_train[idx]
        
        for k in range(x_train.shape[0]//batch_size):
            
            if k==0:
                loss_temp = []
                
            x_batch = x_epoch[k*batch_size:(k+1)*batch_size]
            
            feed_dict = {x : x_batch}
            
            loss_run, _ = sess.run([loss, train], feed_dict=feed_dict)
            
            loss_temp.append(loss_run) 
        
        loss_avg = np.array(loss_temp).mean()
        loss_trace.append(loss_avg)
        print(f'{i+1}/{epoch} loss {loss_avg}')
    
    feed_dict = {x : x_test}
    x_test_recon, x_test_hidden = sess.run([x_recon, hidden], 
                                           feed_dict=feed_dict)
    
    x_grid = np.linspace(0.01, 0.99, 20)
    y_grid = np.linspace(0.01, 0.99, 20)
    X_grid, Y_grid = np.meshgrid(x_grid, y_grid)
    X_grid, Y_grid = X_grid.reshape((-1,1)), Y_grid.reshape((-1,1))
    hidden_grid = np.concatenate([X_grid, Y_grid], axis=1)
    
    feed_dict = {hidden : hidden_grid}
    latent_space_walk = sess.run(x_recon, feed_dict=feed_dict)

1/2000 loss 0.2947521507740021
2/2000 loss 0.23454144597053528
3/2000 loss 0.2232358306646347
4/2000 loss 0.217111736536026
5/2000 loss 0.21164637804031372
6/2000 loss 0.20700182020664215
7/2000 loss 0.20387592911720276
8/2000 loss 0.20149438083171844
9/2000 loss 0.199406698346138
10/2000 loss 0.1974521428346634
11/2000 loss 0.1957947015762329
12/2000 loss 0.19429795444011688
13/2000 loss 0.1928120106458664
14/2000 loss 0.1915464550256729
15/2000 loss 0.19042231142520905
16/2000 loss 0.1894545704126358
17/2000 loss 0.1885395348072052
18/2000 loss 0.18786434829235077
19/2000 loss 0.18702572584152222
20/2000 loss 0.18642359972000122
21/2000 loss 0.1857375055551529
22/2000 loss 0.18519282341003418
23/2000 loss 0.18460217118263245
24/2000 loss 0.18417587876319885
25/2000 loss 0.1836283802986145
26/2000 loss 0.18311582505702972
27/2000 loss 0.18258346617221832
28/2000 loss 0.18216891586780548
29/2000 loss 0.1817370057106018
30/2000 loss 0.18157772719860077
31/2000 loss 0.1809999793767929
32

248/2000 loss 0.16268230974674225
249/2000 loss 0.16283223032951355
250/2000 loss 0.16263550519943237
251/2000 loss 0.16264468431472778
252/2000 loss 0.16301114857196808
253/2000 loss 0.16263775527477264
254/2000 loss 0.16297416388988495
255/2000 loss 0.16276507079601288
256/2000 loss 0.1629081815481186
257/2000 loss 0.16249217092990875
258/2000 loss 0.16243229806423187
259/2000 loss 0.16238684952259064
260/2000 loss 0.16263996064662933
261/2000 loss 0.16261593997478485
262/2000 loss 0.16250211000442505
263/2000 loss 0.16235078871250153
264/2000 loss 0.16245263814926147
265/2000 loss 0.16241279244422913
266/2000 loss 0.16228991746902466
267/2000 loss 0.16212424635887146
268/2000 loss 0.16237600147724152
269/2000 loss 0.16218361258506775
270/2000 loss 0.1621215045452118
271/2000 loss 0.16269850730895996
272/2000 loss 0.16248492896556854
273/2000 loss 0.16244938969612122
274/2000 loss 0.16204731166362762
275/2000 loss 0.16242630779743195
276/2000 loss 0.1620132476091385
277/2000 loss 0.1

492/2000 loss 0.15792188048362732
493/2000 loss 0.15806065499782562
494/2000 loss 0.15804670751094818
495/2000 loss 0.15788887441158295
496/2000 loss 0.1582334190607071
497/2000 loss 0.15809696912765503
498/2000 loss 0.15810658037662506
499/2000 loss 0.15808232128620148
500/2000 loss 0.15777873992919922
501/2000 loss 0.15808933973312378
502/2000 loss 0.1580369621515274
503/2000 loss 0.15775908529758453
504/2000 loss 0.1578272432088852
505/2000 loss 0.15799468755722046
506/2000 loss 0.15780678391456604
507/2000 loss 0.15771058201789856
508/2000 loss 0.15780016779899597
509/2000 loss 0.1579771339893341
510/2000 loss 0.15790720283985138
511/2000 loss 0.15781496465206146
512/2000 loss 0.1577218621969223
513/2000 loss 0.15803194046020508
514/2000 loss 0.15806947648525238
515/2000 loss 0.15790916979312897
516/2000 loss 0.15819977223873138
517/2000 loss 0.15756621956825256
518/2000 loss 0.15762512385845184
519/2000 loss 0.1575772911310196
520/2000 loss 0.15790972113609314
521/2000 loss 0.1577

736/2000 loss 0.1548958122730255
737/2000 loss 0.1550607830286026
738/2000 loss 0.1547728031873703
739/2000 loss 0.15474310517311096
740/2000 loss 0.15464788675308228
741/2000 loss 0.1547222137451172
742/2000 loss 0.15483327209949493
743/2000 loss 0.15535204112529755
744/2000 loss 0.15540266036987305
745/2000 loss 0.15507568418979645
746/2000 loss 0.15475434064865112
747/2000 loss 0.15465185046195984
748/2000 loss 0.15462440252304077
749/2000 loss 0.15470309555530548
750/2000 loss 0.1546904444694519
751/2000 loss 0.1546412855386734
752/2000 loss 0.15456780791282654
753/2000 loss 0.15471959114074707
754/2000 loss 0.1549196094274521
755/2000 loss 0.1546487808227539
756/2000 loss 0.1545306146144867
757/2000 loss 0.1544799655675888
758/2000 loss 0.15446025133132935
759/2000 loss 0.15468262135982513
760/2000 loss 0.1544666290283203
761/2000 loss 0.1545323133468628
762/2000 loss 0.15435704588890076
763/2000 loss 0.15446315705776215
764/2000 loss 0.15462623536586761
765/2000 loss 0.1545124202

980/2000 loss 0.1526913195848465
981/2000 loss 0.1527189314365387
982/2000 loss 0.15236704051494598
983/2000 loss 0.15225735306739807
984/2000 loss 0.1520247906446457
985/2000 loss 0.15219281613826752
986/2000 loss 0.15207408368587494
987/2000 loss 0.15211158990859985
988/2000 loss 0.15205760300159454
989/2000 loss 0.15207326412200928
990/2000 loss 0.15209025144577026
991/2000 loss 0.15200619399547577
992/2000 loss 0.15217046439647675
993/2000 loss 0.15196336805820465
994/2000 loss 0.15188336372375488
995/2000 loss 0.1519422084093094
996/2000 loss 0.1518634855747223
997/2000 loss 0.15198476612567902
998/2000 loss 0.15180565416812897
999/2000 loss 0.15195778012275696
1000/2000 loss 0.15179644525051117
1001/2000 loss 0.15180324018001556
1002/2000 loss 0.15180011093616486
1003/2000 loss 0.15168093144893646
1004/2000 loss 0.15179724991321564
1005/2000 loss 0.1516202986240387
1006/2000 loss 0.15186583995819092
1007/2000 loss 0.15176352858543396
1008/2000 loss 0.15166260302066803
1009/2000 l

1218/2000 loss 0.14978213608264923
1219/2000 loss 0.1498749554157257
1220/2000 loss 0.1497749537229538
1221/2000 loss 0.14974571764469147
1222/2000 loss 0.14972898364067078
1223/2000 loss 0.14995016157627106
1224/2000 loss 0.14957068860530853
1225/2000 loss 0.14962151646614075
1226/2000 loss 0.14981473982334137
1227/2000 loss 0.14977267384529114
1228/2000 loss 0.14981681108474731
1229/2000 loss 0.14959387481212616
1230/2000 loss 0.14961886405944824
1231/2000 loss 0.14962348341941833
1232/2000 loss 0.1495644748210907
1233/2000 loss 0.14964568614959717
1234/2000 loss 0.14950983226299286
1235/2000 loss 0.15034359693527222
1236/2000 loss 0.14985324442386627
1237/2000 loss 0.14958946406841278
1238/2000 loss 0.14961552619934082
1239/2000 loss 0.14968015253543854
1240/2000 loss 0.1498199999332428
1241/2000 loss 0.14952298998832703
1242/2000 loss 0.14968961477279663
1243/2000 loss 0.1494750678539276
1244/2000 loss 0.1494947373867035
1245/2000 loss 0.14966891705989838
1246/2000 loss 0.149446561

1454/2000 loss 0.14798812568187714
1455/2000 loss 0.14788667857646942
1456/2000 loss 0.1476326286792755
1457/2000 loss 0.14776791632175446
1458/2000 loss 0.14768144488334656
1459/2000 loss 0.14767056703567505
1460/2000 loss 0.14783881604671478
1461/2000 loss 0.14770722389221191
1462/2000 loss 0.14776714146137238
1463/2000 loss 0.14781714975833893
1464/2000 loss 0.14780566096305847
1465/2000 loss 0.14786241948604584
1466/2000 loss 0.14777976274490356
1467/2000 loss 0.14759086072444916
1468/2000 loss 0.14799535274505615


In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_trace)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=2, ncols=10, figsize=(20,5))

for i in range(2):
    for j in range(10):
        
        if i==0:
            img = x_test[j].reshape((28,28))
            title = 'original'
        else:
            img = x_test_recon[j].reshape((28,28))
            title = 'recon'
            
        axes[i, j].imshow(img, cmap='binary')
        axes[i, j].set_title(title, fontsize=20)
        axes[i, j].axis('off')
            
plt.tight_layout()

plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(20,10))

for i in range(2):
    for j in range(5):
        
        k = i*5 + j
    
        x_fig = x_test_hidden[y_test==k][:,0]
        y_fig = x_test_hidden[y_test==k][:,1]
        axes[i, j].plot(x_fig, y_fig, '.')
        axes[i, j].set_title(f'label {k}', fontsize=20)
        axes[i, j].set_xlim([0., 1.])
        axes[i, j].set_ylim([0., 1.])
            
plt.show()

In [ ]:
import matplotlib.pyplot as plt

colors = ['r','g','b','m','c','k','gray','silver','rosybrown','indianred']
for i in range(2):
    for j in range(5):
        
        k = i*5 + j
    
        x_fig = x_test_hidden[y_test==k][:,0]
        y_fig = x_test_hidden[y_test==k][:,1]
        plt.plot(x_fig, y_fig, '.', color=colors[k], label=f'label {k}')
        
plt.legend()
plt.xlim([0., 1.])
plt.ylim([0., 1.])
            
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=10, ncols=10, figsize=(40,40))

colors = ['r','g','b','m','c','k','gray','silver','rosybrown','indianred']
for i in range(10):
    for j in range(10):
    
        x_fig = x_test_hidden[y_test==i][:,0]
        y_fig = x_test_hidden[y_test==i][:,1]
        axes[i, j].plot(x_fig, y_fig, '.', color=colors[i])
        
        x_fig = x_test_hidden[y_test==j][:,0]
        y_fig = x_test_hidden[y_test==j][:,1]
        axes[i, j].plot(x_fig, y_fig, '.', color=colors[j])

        axes[i, j].set_xlim([0., 1.])
        axes[i, j].set_ylim([0., 1.])

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=20, ncols=20, figsize=(40,40))

for i in range(20):
    for j in range(20):
        
        img = latent_space_walk[i*20+j].reshape((28,28))
            
        axes[i, j].imshow(img, cmap='binary')
        axes[i, j].axis('off')
            
plt.tight_layout()

plt.show()